In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform, loguniform
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
import itertools

In [ ]:
df_train = pd.read_csv('blood_samples.csv')
X = df_train.drop('Disease', axis=1)  # Caracteristicas
y = df_train['Disease']  # Etiquetas
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

In [ ]:
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

def evaluate_svm(x_train, y_train, x_test, y_test, kernel='poly',do_cross_validation=True):

   model = svm.SVC(kernel=kernel)
   model.fit(x_train, y_train)

   predictions = model.predict(x_test)

   print("Reporte de clasificación:\n", classification_report(y_test, predictions))

   if do_cross_validation:
      cv = cross_val_score(model, X, y, cv=5, scoring='accuracy') #Numero de iteraciones = 5

      print("Resultados de validación cruzada:")
      for i, accuracy in enumerate(cv):
         print(f"Iteración {i+1}: Accuracy = {accuracy}")

      print(f"Accuracy promedio: {np.mean(cv)}")
      print(f"Desviación estándar del accuracy: {np.std(cv)}")

   cm = confusion_matrix(y_test, predictions, labels = model.classes_)
   disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = model.classes_)
   disp.plot()
   plt.show()

In [ ]:
evaluate_svm(x_train, y_train, x_test, y_test, kernel='linear', do_cross_validation=False)

In [ ]:
evaluate_svm(x_train, y_train, x_test, y_test, kernel='poly', do_cross_validation=True)

In [ ]:
evaluate_svm(x_train, y_train, x_test, y_test, kernel='rbf', do_cross_validation=False)


El código optimiza, entrena y evalúa un modelo de Máquinas de Vectores de Soporte (SVM) con visualización comparativa de la precisión para diferentes parámetros. Se realizan los siguientes pasos:

Ajuste de hiperparámetros (Randomized Search CV):

Se define un espacio de búsqueda para los hiperparámetros del SVM (kernel, C, gamma, etc.).
El modelo SVM se entrena con diferentes combinaciones de hiperparámetros utilizando validación cruzada y se evalúa su precisión.
Se encuentra la mejor configuración de hiperparámetros según la mayor precisión media.

Entrenamiento y evaluación:

Se crea un modelo SVM final con los mejores hiperparámetros.
El modelo final se entrena en los datos de entrenamiento.
Se realizan predicciones en los datos de prueba.
Se evalúa el rendimiento del modelo mediante un informe de clasificación y una matriz de confusión.

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

    param_distributions = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': loguniform(0.001, 100.0),
        'gamma': loguniform(0.00001, 1.0),
        'degree': [2, 3, 4, 5],
        'coef0': uniform(-1.0, 1.0),
        'shrinking': [True, False],
        'probability': [True, False],
        'tol': loguniform(1e-6, 1e-2),
        'cache_size': [100, 200, 500],
        'class_weight': ['balanced', None],
        'verbose': [0],
        'max_iter': [1000, 2000, 5000],
        'decision_function_shape': ['ovo', 'ovr'],
    }

    model = svm.SVC()

    random_search = RandomizedSearchCV(model, param_distributions, n_iter=100, cv=5, scoring='accuracy')

    random_search.fit(x_train, y_train)

    best_params = random_search.best_params_

    print("Mejores parámetros:", best_params)
    print("Mejor puntaje:", random_search.best_score_)

    best_model = svm.SVC(**best_params)
    best_model.fit(x_train, y_train)

    predictions = best_model.predict(x_test)
    print("Reporte de clasificación:\n", classification_report(y_test, predictions))
    cm = confusion_matrix(y_test, predictions, labels = best_model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels = best_model.classes_)
    disp.plot()
    plt.show()

Ploteamos los distintos parametros en relaccion a su accuracy, pero como RandomizedSearchCV mezcla los parametros de manera aleatoria, los graficos no tendran una relaccion consistente.

In [ ]:
df = pd.DataFrame(random_search.cv_results_)

params_of_interest = ['param_kernel', 'param_C', 'param_gamma', 'param_degree']

for param in params_of_interest:
    plt.figure(figsize=(10, 6))
    # Filtrar el DataFrame para eliminar las filas donde el parámetro es None
    filtered_df = df[df[param].notna()]
    plt.plot(filtered_df[param], filtered_df['mean_test_score'], 'o')
    plt.xlabel(param)
    plt.ylabel('Mean Test Score')
    plt.show()